# DATA PREPARATION STARTING FROM DATAFRAMES

In [1]:
import itertools
import pathlib
import pickle as pkl

import numpy as np
import pandas as pd
# import torch

In [2]:
datestamp = "20241030-125950"
sim_folder = "/mnt/c/Users/nikko/OneDrive/Documents/Uni/magistrale/tesi/simulations"

#Directory defining
sim_folder = pathlib.Path(sim_folder)
datafolder = sim_folder / datestamp / "data"

In [3]:
path_list = []
for path in datafolder.iterdir():
    if path.is_dir():
        num = path.name[3:]
        df_path = path / f"{datestamp}_run{num}.txt"
        path_list.append(df_path)

In [4]:
len(path_list)

13

In [5]:
df_list = []
for path in path_list:
    df_list.append(pd.read_csv(path))

In [6]:
timestep = 1e-5
if len(np.unique(df_list[0].Time)) != max(df_list[0].Time):
    timestep = timestep*(max(df_list[0].Time)-1)/(len(np.unique(df_list[0].Time))-1)

## Velocity calc.

I am dropping the first instant for velocity is obv. NaN. 

In [7]:
for df in df_list:
    df.sort_values(by=["N","Time"],inplace=True)
    df[["vx","vy"]] = df.groupby("N")[["xpos","ypos"]].diff()/timestep #velocity calculation
    df.dropna(inplace=True)
    df["Time"]=df["Time"]-1

In [16]:
num_exp = len(df_list)
Np = df_list[0].N.max()
num_steps = len(np.unique(df_list[0].Time))

## Position reshaping

In [9]:
ppos = []
for df in df_list:
    Np = df.N.max()
    x = np.array(df.xpos).reshape(Np,-1)
    y = np.array(df.ypos).reshape(Np,-1)
    xy = np.stack([x,y],axis=2)
    ppos.append(xy)
# position = np.concatenate(ppos,axis=1)
position = np.stack(ppos,axis=3)
position = np.transpose(position,(3,1,0,2))


## Velocity reshaping

In [10]:
vv = []
for df in df_list:
    vx = np.array(df.vx).reshape(Np,-1)
    vy = np.array(df.vy).reshape(Np,-1)
    vxy = np.stack([vx,vy],axis=2)
    vv.append(vxy)
vel = np.stack(vv,axis=3)
vel = np.transpose(vel,(3,1,0,2))

## Force

In [11]:
force_ = []
for df in df_list:
    Np = df.N.max()
    fx = np.array(df.fx).reshape(Np,-1)
    fy = np.array(df.fy).reshape(Np,-1)
    fxy = np.stack([fx,fy],axis=2)
    force_.append(fxy)
# position = np.concatenate(ppos,axis=1)
force = np.stack(force_,axis=3)
force = np.transpose(force,(3,1,0,2))

## Orientation

In [65]:
angle_ = []
for df in df_list:
    Np = df.N.max()
    theta = np.array(df.orientation).reshape(Np,-1)
    angle_.append(theta)
angle = np.stack(angle_, axis=2)[:,np.newaxis]
angle = np.transpose(angle,(3,2,0,1))

## Drag Coefficient

In [18]:
R = 2.
gamma = 6*np.pi*R*1e-3
gamma = np.repeat(gamma, Np)
gamma = gamma[:,np.newaxis]
gamma = np.tile(gamma,(num_exp,num_steps,1,1))

## Edges

In [25]:
edges = list(itertools.combinations(range(Np), 2))
edges = np.array(edges)

In [26]:
data = {
    "position": position,
    "velocity": vel,
    "drag_coefficient": gamma,
    "edge_list": edges,
}

In [27]:
with open('data.pkl', '+rb') as f:
    pkl.dump(data, f)